In [ ]:
import pandas as pd
import uproot
import uproot_methods
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.colors import LightSource
%matplotlib inline
import mplhep as hep
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
from mpl_toolkits.axes_grid1 import make_axes_locatable

plt.style.use(hep.style.ROOT) # For now ROOT defaults to CMS

import numpy as np
from scipy.stats import norm
from scipy.optimize import curve_fit
from scipy import interpolate

import itertools

import functions_giammi as fugi
cmap_temp, cmap_temp_go, Magma_r, Seismic_r = fugi.customcmaps()

dir = r"../PYTHON_graphs/OUTPUTS/N2_calibration/"
cmap_viridis = mpl.cm.get_cmap('viridis')


In [ ]:
N2 = uproot.open(r"../PYTHON_graphs/DATA/Experiments/N2_420eV_CR_analysis_N+N+_linear_Arstretch_newE116vpcm_newt0.root")
# N2.keys()

In [ ]:
#coincidence MFPAD
phi_mol1D,MFPAD=fugi.import_TH1Dgeneric(N2,"N+N+/mol_coincidence/elec_hit_0/mol_frame/in_polarization_plane/MFPAD")
MFPADerr=N2["N+N+/mol_coincidence/elec_hit_0/mol_frame/in_polarization_plane/MFPAD"].errors()
phi_mol2D,ctheta_mol,MFPAD3D=fugi.import_TH2Dgeneric(N2,"N+N+/mol_coincidence/elec_hit_0/mol_frame/in_polarization_plane/MFPAD3D")

#break-up channels
en,KER=fugi.import_TH1Dgeneric(N2,"N+N+/mol_ion/KER")

#rel momenta
p_x,p_y,p_rel_xy=fugi.import_TH2Dgeneric(N2,"N+N+/mol_ion/momenta/p_relx vs p_rely")
p_y,p_z,p_rel_yz=fugi.import_TH2Dgeneric(N2,"N+N+/mol_ion/momenta/p_rely vs p_relz")
p_x,p_z,p_rel_xz=fugi.import_TH2Dgeneric(N2,"N+N+/mol_ion/momenta/p_relx vs p_relz")

p_ysum,p_zsum,p_sum_yz=fugi.import_TH2Dgeneric(N2,"N+N+/mol_ion/momenta/p_sumy vs p_sumz")

#ion momenta
p1_x,p2_x,p12_x=fugi.import_TH2Dgeneric(N2,"N+N+/mol_ion/momenta/ion1_x vs ion2_x")
p1_y,p2_y,p12_y=fugi.import_TH2Dgeneric(N2,"N+N+/mol_ion/momenta/ion1_y vs ion2_y")
p1_z,p2_z,p12_z=fugi.import_TH2Dgeneric(N2,"N+N+/mol_ion/momenta/ion1_z vs ion2_z")



In [ ]:
d = np.linspace(10, 11, 50)
KER_masked = np.ma.masked_array(KER,[(en <= 9.8) | (en >= 10.7)] ).compressed()
en_masked = np.ma.masked_array(en,[(en <= 9.8) | (en >= 10.7)] ).compressed()

mean = sum(en_masked*KER_masked)/sum(KER_masked)
sigma = np.sqrt(sum(KER_masked*(en_masked-mean)**2)/sum(KER_masked))

def gaus_fit(x,a,x0,sigma):
    return a*np.exp(-(x-x0)**2/(2*sigma**2))

popt,pcov = curve_fit(gaus_fit,en_masked,KER_masked,p0=[max(KER_masked), mean, sigma])
fit_info = "Gaussian fit \n\u00B5 = %.2f \nstd = %.2f" % (mean, sigma)
# fit_info = (f"Gauss fit: \u00B5 = {mu:+.2f}, \u00b2 std = {std:+.2f}")

fig,ax= plt.subplots(1, figsize=(12, 10))

ax.plot(en,KER, color=cmap_viridis(0))
ax.plot(en_masked,gaus_fit(en_masked,*popt), color=cmap_viridis(0.3), linewidth=5, alpha = 0.6)
ax.legend(title=fit_info, loc="best")


# ax.set_title("fish fillet y-axis")

ax.set_xlabel('energy [eV]')
ax.set_xlim([5, 14])
ax.set_ylabel('counts')
ax.ticklabel_format(axis="y",style="sci",scilimits=(0,1))
ax.set_ylim([0, 55000])
ax.grid()

fig.savefig(dir+"KER_gaus.png", dpi=300, transparent=False)
fig.savefig(dir+"KER_gaus.svg", dpi=300, transparent=False)

In [ ]:
tck = interpolate.splrep(phi_mol1D, MFPAD,s=0)
# xnew = np.arange(0, 2*np.pi, np.pi/50)
xnew = np.arange(-180, 180, 0.5)
ynew = interpolate.splev(xnew, tck)

fig, ax = plt.subplots(subplot_kw={'projection': 'polar'})
# plt.scatter(phi_mol1D/180.*np.pi, MFPAD, color=cmap_viridis(0), s=80)
plt.errorbar(phi_mol1D/180.*np.pi, MFPAD, yerr=MFPADerr, fmt="o", color=cmap_viridis(0), lw= 20, mec='green', ms=7)
plt.plot(xnew/180.*np.pi, ynew, c="r", alpha=0.5)
ax.set_rticks([2000,4000,6000, 8000])  # Less radial ticks
ax.set_yticklabels([])
# ax.set_rti  # Move radial labels away from plotted line
ax.grid(True)

fig.savefig(dir+"MFPAD_err.png", dpi=300, transparent=False)
fig.savefig(dir+"MFPAD_err.svg", dpi=300, transparent=False)

In [ ]:
#3D MFPAD
from scipy.ndimage.filters import gaussian_filter
Yn, Xn = np.meshgrid(ctheta_mol, phi_mol2D)

fig,ax= plt.subplots(1, figsize=(12, 10))
aspect=(phi_mol1D.max()/ctheta_mol.max())
# cs=ax.contourf(phi_mol2D,ctheta_mol,MFPAD3D.T,cmap=cmap_temp)
cs=ax.pcolormesh(phi_mol2D,ctheta_mol,MFPAD3D.T,shading='gouraud', cmap=cmap_temp)
ax.contour(Xn, Yn, gaussian_filter(MFPAD3D, 1.), 15, colors='k', alpha=0.15)
#too much computational intensive
# cs,ax=fugi.plot_interpolation(phi_mol2D,ctheta_mol,MFPAD3D.T,ax=ax, xstep=1, ystep=0.05, cmap=cmap_temp)

# ax.set_title("fish fillet y-axis")

ax.set_xlabel('phi [DEG]')
ax.set_ylabel('cos(theta) [adm]')
ax.set_aspect(aspect)

cbar = fig.colorbar(cs,shrink=0.9, ax=ax)
cbar.set_ticks([MFPAD3D.min(),MFPAD3D.max()])
cbar.set_ticklabels(["min","max"])
cbar.ax.set_ylabel('normalized counts')

fig.savefig(dir+"MFPAD3d.png", dpi=300, transparent=False)
# fig.savefig(dir+"MFPAD3d.svg", dpi=300, transparent=False)

In [ ]:
#prel
fig,(ax1,ax2,ax3)= plt.subplots(1, 3, figsize=(20, 10), constrained_layout=True)
fig.suptitle("Relative momenta $N^{+}-N^{+}$ multiple axis", y=0.85)

aspect=(p_x.max()/p_y.max())

atot =[ax1,ax2,ax3]

# cs1=ax1.contourf(p_x,p_y,np.ma.masked_array(p_rel_xy.T,p_rel_xy.T<3))
# cs2=ax2.contourf(p_x,p_z,np.ma.masked_array(p_rel_xz.T,p_rel_xz.T<3))
# cs3=ax3.contourf(p_y,p_z,np.ma.masked_array(p_rel_yz.T,p_rel_yz.T<3))

cs1=ax1.pcolormesh(p_x,p_y,np.ma.masked_array(p_rel_xy.T,p_rel_xy.T<3), shading='gouraud')
cs2=ax2.pcolormesh(p_x,p_z,np.ma.masked_array(p_rel_xz.T,p_rel_xz.T<3), shading='gouraud')
cs3=ax3.pcolormesh(p_y,p_z,np.ma.masked_array(p_rel_yz.T,p_rel_yz.T<3), shading='gouraud')

ax1.set_xlabel(r"relative $momentum_{x}$ [a.u.]")
ax1.set_ylabel(r"relative $momentum_{y}$ [a.u.]")
ax2.set_xlabel(r"relative $momentum_{z}$ [a.u.]")
ax2.set_ylabel(r"relative $momentum_{x}$ [a.u.]")
ax3.set_xlabel(r"relative $momentum_{z}$ [a.u.]")
ax3.set_ylabel(r"relative $momentum_{y}$ [a.u.]")

for a in atot:
    a.set_aspect(aspect)
    a.set_xlim([-120., 120.])
    a.set_ylim([-120., 120.])
    # a.grid()

cax = fig.add_axes([1.02, 0.20, 0.01, 0.58])
cbar = fig.colorbar(cs3,shrink=0.9, cax=cax)
cbar.set_ticks([np.ma.masked_array(p_rel_xy.T,p_rel_xy.T<3).min(),p_rel_xz.max()])
cbar.set_ticklabels(["min","max"])
cbar.ax.set_ylabel('normalized counts')

fig.savefig(dir+"prel_N-N.png", bbox_inches='tight', dpi=300, transparent=False)
# fig.savefig(dir+"prel_N-N.svg", bbox_inches='tight', dpi=300, transparent=False)

In [ ]:
#ion relative momenta
fig,(ax1,ax2,ax3)= plt.subplots(1, 3, figsize=(20, 10), constrained_layout=True)
# fig,(ax1,ax2,ax3)= plt.subplots(1, 3, figsize=(20, 10))
fig.suptitle("Ion momenta $N^{+}-N^{+}$ multiple axis", y=0.85)

aspect=(p_x.max()/p_y.max())

atot =[ax1,ax2,ax3]

# cs1=ax1.contourf(p1_x,p2_x,np.ma.masked_array(p12_x.T,p12_x.T<3))
# cs2=ax2.contourf(p1_y,p2_y,np.ma.masked_array(p12_y.T,p12_y.T<3))
# cs3=ax3.contourf(p1_z,p2_z,np.ma.masked_array(p12_z.T,p12_z.T<3))

cs1=ax1.pcolormesh(p1_x,p2_x,np.ma.masked_array(p12_x.T,p12_x.T<3), shading='gouraud')
cs2=ax2.pcolormesh(p1_y,p2_y,np.ma.masked_array(p12_y.T,p12_y.T<3), shading='gouraud')
cs3=ax3.pcolormesh(p1_z,p2_z,np.ma.masked_array(p12_z.T,p12_z.T<3), shading='gouraud')

ax1.set_xlabel(r"$ion1_{x}$ [a.u.]")
ax1.set_ylabel(r"$ion2_{x}$ [a.u.]")
ax2.set_xlabel(r"$ion1_{y}$ [a.u.]")
ax2.set_ylabel(r"$ion2_{y}$ [a.u.]")
ax3.set_xlabel(r"$ion1_{z}$ [a.u.]")
ax3.set_ylabel(r"$ion2_{z}$ [a.u.]")

for a in atot:
    a.set_aspect(aspect)
    a.set_xlim([-140., 140.])
    a.set_ylim([-140., 140.])
    # a.grid()
    a.axline((0, 0), (-1, 1), color='black', linestyle='--', alpha=0.5)
    
# divider = make_axes_locatable(plt.gca())
# cax = divider.append_axes("right", "5%", pad="3%")

cax = fig.add_axes([1.02, 0.20, 0.01, 0.58])
cbar = fig.colorbar(cs3,shrink=0.9, cax=cax)
cbar.set_ticks([np.ma.masked_array(p12_z.T,p12_z.T<3).min(),p12_z.max()])
cbar.set_ticklabels(["min","max"])
cbar.ax.set_ylabel('normalized counts')

fig.savefig(dir+"ionmom_N-N.png", bbox_inches='tight', dpi=300, transparent=False)
# fig.savefig(dir+"ionmom_N-N.svg", bbox_inches='tight', dpi=300, transparent=False)

In [ ]:
import matplotlib.animation as animation

#cartesian product of unique values, (48*36) each
cosphi_MFPAD = np.array(list(itertools.product(ctheta_mol, phi_mol2D)))

ctheta_full = np.array([col[0] for col in cosphi_MFPAD])
theta_rad = np.arccos(ctheta_full)

phi_full = np.array([col[1] for col in cosphi_MFPAD])
phi_rad = phi_full * np.pi/180.

# convertion to cartesian coordinates 1D vectors of shape (2000,1)
x = MFPAD3D.T.reshape(-1) * np.sin(theta_rad) * np.cos(phi_rad)
y = MFPAD3D.T.reshape(-1) * np.sin(theta_rad) * np.sin(phi_rad)
z = MFPAD3D.T.reshape(-1) * ctheta_full

# distance from the origin for the cmap
# remember do not use math, but np for fast operations on vectors
d = np.sqrt(x**2+y**2+z**2)

# reshaping to have the right dimension for 3D
X = np.reshape(x, (48, 36))
Y = np.reshape(y, (48, 36))
Z = np.reshape(z, (48, 36))

d_matrix = np.sqrt(X**2+Y**2+Z**2)

fig = plt.figure(figsize=(20,20))
# fig, ax = plt.subplots(subplot_kw={"projection": "3d"})
ax = fig.add_subplot(111, projection = "3d", )

# ax.scatter(x, y, z, c=d, cmap='viridis', alpha=0.4) # X,Y,Z 1D vectors -> everything ok
# ax.scatter(x, y, z, c=d, cmap='viridis' , s=40) # X,Y,Z 1D vectors -> everything ok

def init():
    ax.scatter(x, y, z, c=d, cmap='viridis' , s=160)
    ax.set_xlabel('x')
    ax.set_xlim3d(x.min(), x.max())
    ax.set_ylabel('y')
    ax.set_ylim3d(y.min(), y.max())
    ax.set_zlabel('z')
    ax.set_zlim3d(z.min(), z.max())
    ax.title.set_text("scattered with contour projections alpha=0.44")

    ## the right sostitution is the ax.plot or ax.fill_between (apparently non supported by 3daxes)
    #ax.contourf(X, Y, Z, zdir='z', alpha=0.2, offset=z.min())
    #ax.contourf(X, Y, Z, zdir='y', alpha=0.2, offset=y.max())
    #ax.contourf(X, Y, Z, zdir='x', alpha=0.2, offset=x.min())
    return fig,

def animate(i):
    ax.view_init(elev=30., azim=3.6*i)
    return fig,

ani = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=100, interval=100, blit=True)  

ani.save("OUTPUTS/N-N.mp4", fps =15, writer="ffmpeg")
# HTML(ani.to_html5_video())
plt.show(False)

In [ ]:
from scipy.spatial import ConvexHull

fig = plt.figure()
# fig, ax = plt.subplots(subplot_kw={"projection": "3d"})
ax1 = fig.add_subplot(111, projection = "3d", )

points_orig = np.vstack((x, y, z)).T
ind = np.argsort(d)
points= np.take_along_axis(points_orig, np.vstack((ind,ind,ind)).T, axis=0)

hull=ConvexHull(points, qhull_options="QJ Pp")

# Plot defining corner points
ax1.plot(points.T[0], points.T[1], points.T[2], "ko")

# 12 = 2 * 6 faces are the simplices (2 simplices per square face)
for s in hull.simplices:
    s = np.append(s, s[0])  # Here we cycle back to the first coordinate
    ax1.plot(points[s, 0], points[s, 1], points[s, 2], "r-")

# Make axis label
for i in ["x", "y", "z"]:
    eval("ax.set_{:s}label('{:s}')".format(i, i))

plt.show()
